In this file I seek to perform a few first experiments on the chatGPT API and then run the same experiment as on the annotated dataset.
- next steps: Calibrate chatGPT prompt
- sample preferred dataset
- label with all models

# before publishing fix API key issue!!!

In [1]:
import pandas as pd
import pickle as pkl
import openai
from sklearn.metrics import confusion_matrix
import time
import json


# import large file
with open('../2-annotated-dataset-experiments/results.pkl', 'rb') as f:
    pre_df = pkl.load(f)

# shorten to not murder my computer
# df = pre_df.iloc[:50]
#del pre_df

# retrieve API key
import os
api_key = os.environ.get('OPENAI_API_KEY')
# print(api_key)

In [2]:
# only execute for small experiments
# only get the ones without disclosures
df = pre_df[pre_df["has_disclosures"]==False]

# sample 30 ads, 30 non-ads
# df = pd.concat([pre_df[pre_df.loc[:,"label"]==True].sample(17),
#                 pre_df[pre_df.loc[:,"label"]==False].sample(17)]).copy()
# df.reset_index(drop=True,inplace=True)
del pre_df

In [28]:
# catch error automatically, print info and wait 2 mins 
completions = []
i=0
for txt in df.loc[:,"caption"]:
    i+=1
    if i%9==0:
      print(f"Counter at {i}")
      time.sleep(65)

    # timer to stay within my quota
    #if i%25==24:
    #    print(f"sleep at {i}") 
    #    time.sleep(65)
    messages=[
              {"role": "system", "content": "You are an assistant helping an academic finding posts that contains (potentially non-commerical) promotional or sponsored material. You are given its caption."},
              {"role": "user", "content": f'I am an academic looking for posts that might have promotional content, even if non-commerical. You will pre-filter a list of posts based on their caption and decide whether my students can ignore it or should have a look at it to sort them manually. If you notice the slightest indication that there might be any chance it could contain a (potentially non-obvious) promotion of a product, service, partnership, even if the promotion is not commercial or not tied to a specific vendor, let humans have a look at by returning "Keep". Return "Keep" even if you are not certain. Always return keep if business accounts are linked or products/services mentioned, even if there is not indication of a partnership. If you think it is very unlikely that it contains (potentially non-obvious) promotion of a product, service, partnership or anything the like and there are no business or specific products mentioned and there is no need for a human to have a look at it, predict ignore. Return either "Keep" or "Ignore". If you are uncertain, err strongly towards "Keep". Also return a brief reasoning \n\nPost: "I love cheeseburgers so much!😱 @barneysburgershop"\nAny pointers to (undisclosed) sponsorship, promotional acitivty etc: Keep \nReasoning: @barneysburgershop is tagged. This might be a business and thus suggests that there is some chance for a paid relationship\n\nPost: "Three days to go until we get to share our big day with you!! #OliviaAndAlex Said Yes, this Friday on @TLCTVUK 🎉"\nAny pointers to (undisclosed) sponsorship, promotional acitivty etc: Keep \nReasoning: The language sounds like promotional activity.\n\nPost: "{txt}"\nAny pointers to undisclosed sponsorship:'}]
    
    try:
      response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=messages)
      completions.append(response["choices"][0]["message"]["content"])
    except Exception as err:
      print("Waiting for 65s", err.__class__.__name__)
      #time.sleep(65)
      #response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
      #                                      messages=messages)
      completions.append(response["choices"][0]["message"]["content"])
    # Save the response text

Waiting for 65s RateLimitError
Counter at 9
Waiting for 65s RateLimitError
Counter at 18
Counter at 27


In [29]:
# Make predictions to booleans
completions_as_boolean = {}
for model in ["0"]:
    completions_as_boolean[model] = [True if response.startswith("Keep") 
                          else False if response.startswith("Ignore")
                          else "warning" for response in completions]

    # check for bad completions
    if "warning" in completions_as_boolean[model]: print("Warning")

    # obtain confusion matrix
    else:
        print(confusion_matrix(df.loc[:,"label"],completions_as_boolean[model]))
        print(confusion_matrix(df.loc[:,"label"],df.loc[:,"baseline"]))
        print(confusion_matrix(df.loc[:,"baseline"],completions_as_boolean[model]))

Warning


In [38]:
completions_as_boolean["0"][20] = False

In [42]:
sum(pd.Series(completions_as_boolean["0"])==True)

16

In [41]:
print(confusion_matrix(df.loc[:,"label"],completions_as_boolean[model]))
print(confusion_matrix(df.loc[:,"label"],df.loc[:,"baseline"]))
print(confusion_matrix(df.loc[:,"baseline"],completions_as_boolean[model]))

[[14  3]
 [ 4 13]]
[[11  6]
 [ 3 14]]
[[14  0]
 [ 4 16]]


In [30]:
completions

['Keep\nReasoning: "@bella_gioianyc" is tagged which suggests that there is a business relationship. The use of "DEVOURPOWER" in the hashtag and promotion of the YouTube channel further indicates potential promotional content.',
 'Keep\nReasoning: @nordstrom is tagged with a call to action to check out the full assortment, suggesting there is a partnership.',
 'Keep\nReasoning: The use of @carolinaherrera and hashtag #GoodGirl suggests a potential partnership and promotional activity.',
 'Keep\nReasoning: The use of "#workingwithchanel" indicates a promotional relationship.',
 'Keep\nReasoning: @proteinworld is mentioned and their product is being promoted to build muscle, indicating a possible sponsored post.',
 'Keep\nReasoning: @proteinworld is mentioned and their product is being promoted to build muscle, indicating a possible sponsored post.',
 'Keep\nReasoning: The post talks about shopping on the @liketoknow.it app, which suggests there may be a paid partnership or affiliation.'

In [6]:
sum(completions_as_boolean["0"])
result = pd.concat([df["baseline"].reset_index(drop=True),df["label"].reset_index(drop=True),pd.Series(completions_as_boolean["0"]).reset_index(drop=True)],axis=1)
sum(result[0])

14

In [ ]:
# Qualitative findings: 
# 1) It is extremely hard to convince it to make it be more lenient with predictions. I suspect this is an induced "fear" of saying something wrong due to the fine-tuning to behave correctly
# 2) If it plans to predict a reasoning _even after_ having done a classification, this increases its likelihood to predict true. This is particularly interesting because it contradicts the comon
# 3) mention in the report that i strongly optimised for finding things and thus the accuracy is lower than it could be
# 4) If the models disagree, it always contains an ad. All shared prediction errors are both models say "False" even though its true

In [12]:
# Generate classifications for davinci only

completions ={"0":[],"1":[]} #{"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}
i=0
for x in range(1):#,"text-ada-001", "text-babbage-001", "text-curie-001"]:
    #i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        #if i%25==24:
        #    print(f"sleep at {i}") 
        #    time.sleep(65)
            
        response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                    {"role": "system", "content": "You are an assistant who gives a recommendation on whether you think it is likely that a post contains ads, given its caption."},
                    {"role": "user", "content":
                        f"Might the following post have been sponsored? \nPost: '{txt}'\nRespond with nothing more than 'True' or 'False':"}
                    ]
                )

        # Save the response text
        completions[str(x)].append(response["choices"][0]["message"]["content"])
    print(str(x)," done")

Counter at 10


RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-6DSDI7PThDmCIofHg69pz46D on requests per min. Limit: 20 / min. Current: 30 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.